In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# The first task is answer the following questions using some of the methods we have covered in the lecture/demo.

# How many houses are in this dataset?
# How many features are there for predicting house price?
# Are there any null values in this dataset?
# Which three variables are best correlated with house price (include correlation coefficients)?
# Which three variables are least correlated with house price (include correlation coefficients)?

In [3]:
#Import seattle data
seattlehprices = pd.read_csv('/Users/morganscafidi/Documents/GitHub/geospatial-data-science/labs/lab5/lab5/seattle_house_prices.csv')

In [4]:
# How many houses are in this dataset? 
# There are 19,451 houses in this dataset.
seattlehprices

,price,bedrooms,bathrooms,sqft_living,sqft_lot,yr_built,lat,long
0,538000,3,2.25,2570,7242,1951,47.7210,-122.319
1,180000,2,1.00,770,10000,1933,47.7379,-122.233
2,604000,4,3.00,1960,5000,1965,47.5208,-122.393
3,510000,3,2.00,1680,8080,1987,47.6168,-122.045
4,1230000,4,4.50,5420,101930,2001,47.6561,-122.005
...,...,...,...,...,...,...,...,...
19446,475000,3,2.50,1310,1294,2008,47.5773,-122.409
19447,360000,3,2.50,1530,1131,2009,47.6993,-122.346
19448,400000,4,2.50,2310,5813,2014,47.5107,-122.362
19449,400000,3,2.50,1600,2388,2004,47.5345,-122.069


In [5]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(seattlehprices, geometry=gpd.points_from_xy(seattlehprices['long'], seattlehprices['lat']))
gdf = gdf.set_crs(4326, allow_override=True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')
coast_utm = coast.to_crs('EPSG:32610')

NameError: name 'coast' is not defined

In [ ]:
# Compute distance to coast
distance_to_coast = []
for i in range(gdf_utm.shape[0]):
    distance_to_coast.append(coast_utm.distance(gdf_utm['geometry'].iloc[i]).min())
    
# Add to DataFrame
gdf_utm['distance_to_coast'] = distance_to_coast

In [6]:
# How many features are there for predicting house price?
# There are 8 existing features for predicting house price.

In [7]:
# Are there any null values in this dataset?
# No there are no null values in the dataset
# Check for NaN values and data types
seattlehprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19451 entries, 0 to 19450
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   price        19451 non-null  int64   
 1   bedrooms     19451 non-null  int64   
 2   bathrooms    19451 non-null  float64 
 3   sqft_living  19451 non-null  int64   
 4   sqft_lot     19451 non-null  int64   
 5   yr_built     19451 non-null  int64   
 6   lat          19451 non-null  float64 
 7   long         19451 non-null  float64 
 8   geometry     19451 non-null  geometry
dtypes: float64(3), geometry(1), int64(5)
memory usage: 1.3 MB


In [8]:
# Compute correlation matrix
corr_matrix = seattlehprices.corr()

# Display just house value correlations
corr_matrix["median_house_value"].sort_values(ascending= False)


KeyError: 'median_house_value'

In [9]:
seattlehprices.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'yr_built',
       'lat', 'long', 'geometry'],
      dtype='object')

In [10]:
#Compute distance to UW Arboretum
#47.63982848173617, -122.29614021004966
from shapely.geometry import Point
geom = Point( -122.28120001226782, 47.6167207495107)
geom_gdf = gpd.GeoDataFrame(geometry = [geom], crs = 4326)
geom_gdf = geom_gdf.to_crs("EPSG:32610")

distance_to_Arboretum = []
for i in range(gdf_utm.shape[0]):
    distance_to_Arboretum.append(geom_gdf.distance(gdf_utm['geometry'].iloc[i]).min())
    
# Add to DataFrame
gdf_utm['distance_to_Arboretum'] = distance_to_Arboretum

In [11]:
#Compute Distance to Bill Gates street
RonMcDon =  Point( -122.23906489686838, 47.61585908737864)
#ParkPoints_gdf = gpd.GeoDataFrame(geometry = [geom], crs = 4326)
RonMcDon_gdf = gpd.GeoDataFrame(geometry = [RonMcDon], crs = 4326)
RonMcDon_gdf = RonMcDon_gdf.to_crs("EPSG:32610")

distance_to_Mcdonald = []
for i in range(gdf_utm.shape[0]):
    distance_to_Mcdonald.append(RonMcDon_gdf.distance(gdf_utm['geometry'].iloc[i]).min())
    
gdf_utm['distance_to_Mcdonald'] = distance_to_Mcdonald

In [12]:
#Compute Distance to Park points
#47.65505093674864, -122.30285131857873
ParkPoints =  [Point(-122.30285131857873, 47.65505093674864),Point(-122.27974971735692, 47.621504321576914),Point(-122.32806723371868, 47.68022234431759), Point(-122.25923189707495,47.67624984969383)]
#ParkPoints_gdf = gpd.GeoDataFrame(geometry = [geom], crs = 4326)
ParkPoints_gdf = gpd.GeoDataFrame(geometry = ParkPoints, crs = 4326)
ParkPoints_gdf = ParkPoints_gdf.to_crs("EPSG:32610")

distance_to_parks = []
for i in range(gdf_utm.shape[0]):
    distance_to_parks.append(ParkPoints_gdf.distance(gdf_utm['geometry'].iloc[i]).min())
    
gdf_utm['distance_to_parks'] = distance_to_parks

In [13]:
gdf_utm.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,yr_built,lat,long,distance_to_Arboretum,distance_to_Mcdonald,distance_to_parks
count,1.945100e+04,19451.000000,19451.000000,19451.000000,1.945100e+04,19451.000000,19451.000000,19451.000000,19451.000000,19451.000000,19451.000000
mean,5.404634e+05,3.369955,2.114943,2081.149967,1.508406e+04,1971.021438,47.559780,-122.213896,17577.839631,17282.067809,15827.153784
std,3.685123e+05,0.906639,0.769818,918.160494,4.160157e+04,29.363143,0.138783,0.140694,10322.279251,9675.053748,11623.873850
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1900.000000,47.155900,-122.519000,67.656932,89.427491,244.415910
25%,3.210000e+05,3.000000,1.750000,1420.000000,5.042500e+03,1952.000000,47.469500,-122.328000,10076.414958,10631.544801,6311.507022
50%,4.500000e+05,3.000000,2.250000,1920.000000,7.620000e+03,1975.000000,47.571700,-122.231000,14706.949034,14349.435795,12642.313741
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.066550e+04,1997.000000,47.677900,-122.125000,23032.664736,21725.538117,22839.251338
max,7.700000e+06,11.000000,8.000000,13540.000000,1.651359e+06,2015.000000,47.777600,-121.315000,73331.928027,70219.092203,70974.897501


In [14]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price                    1.000000
sqft_living              0.702296
bathrooms                0.524395
bedrooms                 0.315804
lat                      0.308082
sqft_lot                 0.090125
yr_built                 0.052453
long                     0.020092
distance_to_parks       -0.333526
distance_to_Arboretum   -0.352885
distance_to_Mcdonald    -0.392472
Name: price, dtype: float64

In [15]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price                    1.000000
sqft_living              0.702296
bathrooms                0.524395
bedrooms                 0.315804
lat                      0.308082
sqft_lot                 0.090125
yr_built                 0.052453
long                     0.020092
distance_to_parks       -0.333526
distance_to_Arboretum   -0.352885
distance_to_Mcdonald    -0.392472
Name: price, dtype: float64

In [16]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price                    1.000000
sqft_living              0.702296
bathrooms                0.524395
bedrooms                 0.315804
lat                      0.308082
sqft_lot                 0.090125
yr_built                 0.052453
long                     0.020092
distance_to_parks       -0.333526
distance_to_Arboretum   -0.352885
distance_to_Mcdonald    -0.392472
Name: price, dtype: float64